# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298532506272                   -0.85    4.9         
  2   -8.300215535398       -2.77       -1.26    1.5    121ms
  3   -8.300438020755       -3.65       -1.89    3.5    156ms
  4   -8.300461562241       -4.63       -2.74    1.9    130ms
  5   -8.300464466672       -5.54       -3.18    2.9    182ms
  6   -8.300464563259       -7.02       -3.37   11.9    323ms
  7   -8.300464605607       -7.37       -3.53    2.2    139ms
  8   -8.300464624067       -7.73       -3.66    1.5    121ms
  9   -8.300464638475       -7.84       -3.85    1.6    128ms
 10   -8.300464641140       -8.57       -3.97    1.1    107ms
 11   -8.300464643797       -8.58       -4.31   10.4    351ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67416211999                   -0.70    6.2         
  2   -16.67858606159       -2.35       -1.14    1.4    323ms
  3   -16.67920564764       -3.21       -1.87    2.5    356ms
  4   -16.67927835576       -4.14       -2.77    3.2    361ms
  5   -16.67928602515       -5.12       -3.19    5.8    540ms
  6   -16.67928620141       -6.75       -3.49    1.9    336ms
  7   -16.67928621816       -7.78       -3.95    1.5    295ms
  8   -16.67928622150       -8.48       -4.59    2.6    342ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32657032039                   -0.56    7.6         
  2   -33.33434970796       -2.11       -1.00    1.9    1.17s
  3   -33.33598993498       -2.79       -1.72    4.6    1.50s
  4   -33.33618277555       -3.71       -2.61    6.4    1.56s
  5   -33.33693967335       -3.12       -2.93    5.2    1.72s
  6   -33.33694312582       -5.46       -3.41    5.4    1.49s
  7   -33.33694376385       -6.20       -4.17    4.0    1.58s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298168622884                   -0.85    5.4         
  2   -8.300241404006       -2.68       -1.59    1.0   91.1ms
  3   -8.300433394642       -3.72       -2.69    3.6    137ms
  4   -8.300442423647       -5.04       -2.58    6.9    214ms
  5   -8.300464315382       -4.66       -3.35    1.1   98.3ms
  6   -8.300464588209       -6.56       -3.79    2.0    122ms
  7   -8.300464642337       -7.27       -4.50    4.2    158ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32466574457                   -0.56    7.4         
  2   -33.32947480975       -2.32       -1.28    1.0    911ms
  3   -27.50135340435   +    0.77       -0.68    6.0    1.69s
  4   -33.12819795968        0.75       -1.39    5.5    1.74s
  5   -33.29676026227       -0.77       -1.43    2.8    1.24s
  6   -33.33431611315       -1.43       -2.10    1.8    1.09s
  7   -33.30524279865   +   -1.54       -1.80    4.2    1.64s
  8   -33.31705460136       -1.93       -1.90    4.8    1.44s
  9   -33.33412761019       -1.77       -2.30    3.1    1.23s
 10   -33.33499453706       -3.06       -2.28    2.1    1.13s
 11   -33.33662610981       -2.79       -2.71    1.8    1.05s
 12   -33.33684568061       -3.66      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.